In [32]:
import pandas as pd
import numpy as np

In [33]:
#Считывание файла с рейтингами фильмов
df = pd.read_excel('Films.xlsx', index_col='User')

In [34]:
#Минимальные и максимальные возможные значения оценкок
min_border = 1
max_border = 5

In [35]:
#Выделение оцененных и неоцененных пользователем target фильмов
target = 'Спицын'
rated_movies = df.columns[np.where(df.loc[target, :] != 0)]
unrated_movies = df.columns[~df.columns.isin(rated_movies)]

In [36]:
#Прогнозирвание оценок, которые пользователь target поставит непросмотренным фильмам 
others = df.drop(target)
p = {}
for unrated in unrated_movies:
    R = []
    dev = []
    for rated in rated_movies:
        rated_both = others.loc[(df[rated]!=0) & (df[unrated]!=0)]
        n = len(rated_both)
        dev.append(((rated_both[unrated]-rated_both[rated])/n).sum())
        R.append(rated)     
    p[unrated] = 1/len(R) * np.add(dev, df.loc[target, rated_movies]).sum()

In [46]:
#Замена оценок ниже 1 на 1 и оценок больше 5 на 5
marks = np.array(list(p.values()))
marks[marks<min_border] = min_border
marks[marks>max_border] = max_border
forecast_df = pd.DataFrame(data={"Фильм": list(p.keys()), "Оценка": marks}).round(1)
forecast_df.set_index("Фильм", inplace=True, drop=True)

In [47]:
print("Прогнозные оценки фильмов для пользователя {}: \n".format(target), forecast_df.sort_values("Оценка", ascending=False))

Прогнозные оценки фильмов для пользователя Спицын: 
                         Оценка
Фильм                         
Хоббит 1                   3.9
Отпуск по обмену           3.2
Любовь и голуби            2.9
Москва слезам не верит     2.9
Легенда №17                1.8
